In [1]:
# Q1.
import pyspark
from pyspark.sql import SparkSession

spark_session = SparkSession.builder.master("local[*]").appName("homework").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/16 17:06:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
pyspark.__version__


'3.3.2'

In [3]:
#Q2

URL = "https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhvhv/fhvhv_tripdata_2021-06.csv.gz"


In [ ]:
# I've ran these below 2 commands in my terminal to get data in my desired location
#1.  mkdir -p data/raw/fhvhv/2021/06
#2. wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhvhv/fhvhv_tripdata_2021-06.csv.gz -O data/raw/fhvhv/2021/06/fhvhv_tripdata_2021_06.csv.gz

# TODO:
# we need to write a pyspark code to read csv data from URL or atleast save data from URL to desired local location

In [4]:
df_fhvhv = spark_session.read.option("header", "true").csv("../notebooks/data/raw/fhvhv/2021/06/")

In [5]:
df_fhvhv.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- dropoff_datetime: string (nullable = true)
 |-- PULocationID: string (nullable = true)
 |-- DOLocationID: string (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [6]:
# we can see that the schema has all strings, so we need to read using pandas to get correct spark alike schema and then 
# use that schema to read data into spark DF again

import pandas as pd

pd_df = pd.read_csv('../notebooks/data/raw/fhvhv/2021/06/fhvhv_tripdata_2021_06.csv.gz', nrows=100) #as we just need schema, we can even read 1 or 10 rows as well



In [7]:
pd_df.dtypes

dispatching_base_num      object
pickup_datetime           object
dropoff_datetime          object
PULocationID               int64
DOLocationID               int64
SR_Flag                   object
Affiliated_base_number    object
dtype: object

In [8]:
pd_df.head()

,dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,SR_Flag,Affiliated_base_number
0,B02764,2021-06-01 00:02:41,2021-06-01 00:07:46,174,18,N,B02764
1,B02764,2021-06-01 00:16:16,2021-06-01 00:21:14,32,254,N,B02764
2,B02764,2021-06-01 00:27:01,2021-06-01 00:42:11,240,127,N,B02764
3,B02764,2021-06-01 00:46:08,2021-06-01 00:53:45,127,235,N,B02764
4,B02510,2021-06-01 00:45:42,2021-06-01 01:03:33,144,146,N,NaN


In [9]:
spark_session.createDataFrame(pd_df).schema

/home/jagadish/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/home/jagadish/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


TypeError: field Affiliated_base_number: Can not merge type <class 'pyspark.sql.types.StringType'> and <class 'pyspark.sql.types.DoubleType'>

In [10]:
import numpy as np

In [11]:
# seems like we have some NaN values in the Affiliated_base_number column
len(pd_df["Affiliated_base_number"].iloc[np.where(pd_df["Affiliated_base_number"].isna() == True)])

34

In [12]:
# so we replace them with the string "NaN"
pd_df["Affiliated_base_number"].iloc[np.where(pd_df["Affiliated_base_number"].isna() == True)] = "NaN"

/tmp/ipykernel_2042/2845800174.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pd_df["Affiliated_base_number"].iloc[np.where(pd_df["Affiliated_base_number"].isna() == True)] = "NaN"


In [13]:
pd_df.head()

,dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,SR_Flag,Affiliated_base_number
0,B02764,2021-06-01 00:02:41,2021-06-01 00:07:46,174,18,N,B02764
1,B02764,2021-06-01 00:16:16,2021-06-01 00:21:14,32,254,N,B02764
2,B02764,2021-06-01 00:27:01,2021-06-01 00:42:11,240,127,N,B02764
3,B02764,2021-06-01 00:46:08,2021-06-01 00:53:45,127,235,N,B02764
4,B02510,2021-06-01 00:45:42,2021-06-01 01:03:33,144,146,N,NaN


In [14]:
spark_session.createDataFrame(pd_df).schema

/home/jagadish/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/home/jagadish/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('SR_Flag', StringType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [15]:
#copy the above schema into some editor eg:vscode and add some spark.sql.types like below
from pyspark.sql import types

fhvhv_schema = types.StructType([types.StructField('dispatching_base_num', types.StringType(), True),
 types.StructField('pickup_datetime', types.TimestampType(), True),
 types.StructField('dropoff_datetime', types.TimestampType(), True),
 types.StructField('PULocationID', types.IntegerType(), True),
 types.StructField('DOLocationID', types.IntegerType(), True),
 types.StructField('SR_Flag', types.StringType(), True),
 types.StructField('Affiliated_base_number', types.StringType(), True)])

In [16]:
input_path = f'../notebooks/data/raw/fhvhv/2021/06/'
output_path = f'../notebooks/data/pq/fhvhv/2021/06/'

df_fhvhv = spark_session.read.option('header', 'true').schema(fhvhv_schema).csv(input_path)

In [17]:
# as asked in the Q2
#Repartition it to 12 partitions and save it to parquet.
df_fhvhv.repartition(12).write.parquet(output_path, mode='overwrite')

In [18]:
ls ../notebooks/data/pq/fhvhv/2021/06/ -l

total 290748
-rw-r--r-- 1 jagadish jagadish        0 Mar 16 17:09 _SUCCESS
-rw-r--r-- 1 jagadish jagadish 24801840 Mar 16 17:08 part-00000-6f96884d-3885-4554-8c89-38f61d8a9890-c000.snappy.parquet
-rw-r--r-- 1 jagadish jagadish 24799764 Mar 16 17:08 part-00001-6f96884d-3885-4554-8c89-38f61d8a9890-c000.snappy.parquet
-rw-r--r-- 1 jagadish jagadish 24813665 Mar 16 17:08 part-00002-6f96884d-3885-4554-8c89-38f61d8a9890-c000.snappy.parquet
-rw-r--r-- 1 jagadish jagadish 24816657 Mar 16 17:08 part-00003-6f96884d-3885-4554-8c89-38f61d8a9890-c000.snappy.parquet
-rw-r--r-- 1 jagadish jagadish 24807133 Mar 16 17:09 part-00004-6f96884d-3885-4554-8c89-38f61d8a9890-c000.snappy.parquet
-rw-r--r-- 1 jagadish jagadish 24800815 Mar 16 17:09 part-00005-6f96884d-3885-4554-8c89-38f61d8a9890-c000.snappy.parquet
-rw-r--r-- 1 jagadish jagadish 24812906 Mar 16 17:09 part-00006-6f96884d-3885-4554-8c89-38f61d8a9890-c000.snappy.parquet
-rw-r--r-- 1 jagadish jagadish 24809398 Mar 16 17:09 part-00007-6f96884d-3885-

In [ ]:
# avg MB size of these partitions are 25MB

In [19]:
# Q3
df_fhvhv.head(5)

[Row(dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 6, 1, 0, 2, 41), dropoff_datetime=datetime.datetime(2021, 6, 1, 0, 7, 46), PULocationID=174, DOLocationID=18, SR_Flag='N', Affiliated_base_number='B02764'),
 Row(dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 6, 1, 0, 16, 16), dropoff_datetime=datetime.datetime(2021, 6, 1, 0, 21, 14), PULocationID=32, DOLocationID=254, SR_Flag='N', Affiliated_base_number='B02764'),
 Row(dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 6, 1, 0, 27, 1), dropoff_datetime=datetime.datetime(2021, 6, 1, 0, 42, 11), PULocationID=240, DOLocationID=127, SR_Flag='N', Affiliated_base_number='B02764'),
 Row(dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 6, 1, 0, 46, 8), dropoff_datetime=datetime.datetime(2021, 6, 1, 0, 53, 45), PULocationID=127, DOLocationID=235, SR_Flag='N', Affiliated_base_number='B02764'),
 Row(dispatching_base_num='B02510', pickup_datetime=datetime.d

In [20]:
df_fhvhv.schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', TimestampType(), True), StructField('dropoff_datetime', TimestampType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('SR_Flag', StringType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [21]:
df_pq_fhvhv = spark_session.read.parquet('../notebooks/data/pq/fhvhv/2021/06/')

In [22]:
df_pq_fhvhv.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B02889|2021-06-04 20:51:44|2021-06-04 21:10:12|         239|         158|      N|                B02889|
|              B02800|2021-06-04 15:50:15|2021-06-04 16:19:29|          75|         116|      N|                  null|
|              B02510|2021-06-02 21:03:38|2021-06-02 21:10:12|         167|         168|      N|                  null|
|              B02867|2021-06-02 12:51:57|2021-06-02 13:05:09|         151|         142|      N|                B02867|
|              B02869|2021-06-21 09:51:45|2021-06-21 10:09:17|         106|          65|      N|                B02869|
|              B02764|2021-06-02 13:27:0

In [25]:
df_results = df_pq_fhvhv.select('dispatching_base_num') \
    .filter((df_pq_fhvhv.pickup_datetime >= '2021-06-15 00:00:00') & (df_pq_fhvhv.pickup_datetime < '2021-06-16 00:00:00'))
# OR the below query also works
# df_results = df_fhvhv.select('dispatching_base_num') \
#     .filter((df_fhvhv.pickup_datetime >= '2021-06-15 00:00:00') & (df_fhvhv.pickup_datetime < '2021-06-16 00:00:00'))

In [26]:
df_results.count()

452470

In [27]:
# Q4
def calculate_trip_duration(pickup_time, dropoff_time):
#     print(dropoff_time, pickup_time)
#     print(dropoff_time-pickup_time)
    diff = (dropoff_time-pickup_time)
    total_seconds = diff.total_seconds() # Convert timedelta into seconds
#     print(total_seconds)
    seconds_in_hour = 3600   # Set the number of seconds in an hour
    td_in_hours = total_seconds / seconds_in_hour # Convert timedelta into hours
#     print(td_in_hours)
    return td_in_hours

In [28]:
from pyspark.sql import functions as F
from pyspark.sql import types

# user defined function
calc_trip_duration = F.udf(calculate_trip_duration, returnType=types.StringType())


In [29]:
df_pq_fhvhv.withColumn('trip_duration', calc_trip_duration(F.to_timestamp(df_pq_fhvhv.pickup_datetime), F.to_timestamp(df_pq_fhvhv.dropoff_datetime))) \
    .select('dispatching_base_num', 'trip_duration') \
    .orderBy(F.col('trip_duration').desc()) \
    .show()

+--------------------+-----------------+
|dispatching_base_num|    trip_duration|
+--------------------+-----------------+
|              B02882|9.966388888888888|
|              B02510|9.966388888888888|
|              B02510|9.637777777777778|
|              B02510|9.624444444444444|
|              B02510|9.480277777777777|
|              B02800|9.471666666666666|
|              B02510|9.402222222222223|
|              B02510|9.393611111111111|
|              B02764|9.376944444444444|
|              B02617|9.365833333333333|
|              B02510|9.246666666666666|
|              B02887|9.106944444444444|
|              B02510|9.056111111111111|
|              B02510|9.030277777777778|
|              B02510|8.817777777777778|
|              B02879|8.774166666666666|
|              B02510|8.758333333333333|
|              B02510|           8.6875|
|              B02510|8.571666666666667|
|              B02510|8.559444444444445|
+--------------------+-----------------+
only showing top

In [30]:
# the above UDF & its custom function are working fine but some logic is missing, so we'll go with the spark inbuilt functions only
# Also, either the spark with SQL or spark with inbuilt functions are better when compared to the spark with UDF because they're costly

In [36]:
df_pq_fhvhv.select(
                F.col("dropoff_datetime"), F.col("pickup_datetime")) \
                .withColumn("diff_hours", (F.col("dropoff_datetime").cast("long") - F.col("pickup_datetime").cast("long"))/3600) \
                .orderBy("diff_hours", ascending=False) \
                .limit(10) \
                .show()

+-------------------+-------------------+------------------+
|   dropoff_datetime|    pickup_datetime|        diff_hours|
+-------------------+-------------------+------------------+
|2021-06-28 08:48:25|2021-06-25 13:55:41|  66.8788888888889|
|2021-06-23 13:42:44|2021-06-22 12:09:45|25.549722222222222|
|2021-06-28 06:31:20|2021-06-27 10:32:29|19.980833333333333|
|2021-06-27 16:49:01|2021-06-26 22:37:11|18.197222222222223|
|2021-06-24 13:08:44|2021-06-23 20:40:43|16.466944444444444|
|2021-06-24 12:19:39|2021-06-23 22:03:31|14.268888888888888|
|2021-06-25 13:05:35|2021-06-24 23:11:00|13.909722222222221|
|2021-06-05 08:36:14|2021-06-04 20:56:02|             11.67|
|2021-06-27 19:07:16|2021-06-27 07:45:19|11.365833333333333|
|2021-06-21 04:04:16|2021-06-20 17:05:12|10.984444444444444|
+-------------------+-------------------+------------------+



In [37]:
# Q6
df_pq_fhvhv.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B02889|2021-06-04 20:51:44|2021-06-04 21:10:12|         239|         158|      N|                B02889|
|              B02800|2021-06-04 15:50:15|2021-06-04 16:19:29|          75|         116|      N|                  null|
|              B02510|2021-06-02 21:03:38|2021-06-02 21:10:12|         167|         168|      N|                  null|
|              B02867|2021-06-02 12:51:57|2021-06-02 13:05:09|         151|         142|      N|                B02867|
|              B02869|2021-06-21 09:51:45|2021-06-21 10:09:17|         106|          65|      N|                B02869|
|              B02764|2021-06-02 13:27:0

In [38]:
zones_df = pd.read_csv('https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv')

In [39]:
zones_df.head()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [40]:
#1 - mkdir -p data/raw/zones
#2 - wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv -O data/raw/zones/taxi_zone_lookup.csv

df_zones = spark_session.read.option('header', 'true').csv('../notebooks/data/raw/zones/taxi_zone_lookup.csv')

In [41]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [42]:
df_pq_fhvhv.registerTempTable('fhvhv_2021_06')

/home/jagadish/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [43]:
df_zones.registerTempTable('zones')

In [44]:
test = spark_session.sql("""
select *
from fhvhv_2021_06 as fhvhv
join zones on fhvhv.PULocationID=zones.LocationID
""")

In [45]:
test.count()

14961892

In [46]:
df_zones.count()

265

In [47]:
df_pq_fhvhv.count()

14961892

In [48]:
results = spark_session.sql("""
select zones.LocationID, count(1) as times_picked, zones.Zone
from fhvhv_2021_06 as fhvhv
join zones on fhvhv.PULocationID=zones.LocationID
group by zones.LocationID, zones.Zone
order by times_picked desc
""")

In [49]:
results.show()

+----------+------------+--------------------+
|LocationID|times_picked|                Zone|
+----------+------------+--------------------+
|        61|      231279| Crown Heights North|
|        79|      221244|        East Village|
|       132|      188867|         JFK Airport|
|        37|      187929|      Bushwick South|
|        76|      186780|       East New York|
|       231|      164344|TriBeCa/Civic Center|
|       138|      161596|   LaGuardia Airport|
|       234|      158937|            Union Sq|
|       249|      154698|        West Village|
|         7|      152493|             Astoria|
|       148|      151020|     Lower East Side|
|        68|      147673|        East Chelsea|
|        42|      146402|Central Harlem North|
|       255|      143683|Williamsburg (Nor...|
|       181|      143594|          Park Slope|
|       225|      141427|  Stuyvesant Heights|
|        48|      139611|        Clinton East|
|       246|      139431|West Chelsea/Huds...|
|        17| 